In [1]:
import pandas as pd
from ipywidgets import *
import plotly.express as px
import ipywidgets as widgets
from PIL import Image
import os,sys
import glob
import cv2
import ipywidgets
import numpy as np
from pathlib import Path
from matplotlib import pylab as plt
import random
from tkinter import Tk, filedialog
import shutil
pd.set_option('display.max_colwidth', None)
from pathlib import Path
from IPython.display import display, clear_output
from ipywidgets import Button
import imageio
import matplotlib
import statistics
from tensorflow.keras.models import Model, load_model, save_model

### <font color ='#27B1CA'>Glioma Classification and Segmentation and NLP analysis </font> ###

This dashboard is used for the detection and classification of Gliomas as well as comparitive analysis between expert labeling and machine labeling to ensure accuracy.

In [2]:
#Corrected Dice Coefficient
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    0

#IOU Coeffifficent
def iou(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [3]:
#Loading Classification Model
model = load_model("C:/Voila/models/brats2019_imagenet_class.h5")

In [4]:
#Loading Segmentation model
model1 = load_model("C:/Voila/models/BraTs2019mri_final_seg.h5", custom_objects={'dice_coef': dice_coef, 'iou':iou, 'precision':precision, 'sensitivity':sensitivity, 'specificity':specificity})

In [5]:
items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

In [6]:
def del_files(src):    #function to delete files from a folder
    files = glob.glob(src)
    for f in files:
        os.remove(f)
        
def del_folder(src):    #function to delete folders
    folders = glob.glob(src)
    for dirs in folders:
        shutil.rmtree(dirs)
    

In [7]:
#Individual Classification Code
def classify(path):
    prediction_list = []
    
    imagesl = glob.glob(path + '/*.png')    
    df = pd.DataFrame({'Image_dir': imagesl})
    for img1 in df['Image_dir']:
        image = cv2.imread(img1)    
        img = cv2.resize(image ,(256, 256))
        img = img / 255
        #img = img[np.newaxis, :, :, 1]
        img = np.expand_dims(img, axis=0)
        pred=model.predict(img)
        mod_pred = np.argmax(pred, axis =1)   
        prediction_list.append(mod_pred[0])
    preds = ["HGG" if i == 0 else "LGG"  if i ==1 else "NT" for i in prediction_list]
    df["Predictions"] = preds
    df["Image_Class"] = df["Image_dir"] + "__" + df["Predictions"]
    t = []
    for images in df["Image_Class"]:
        pth = Path(images)
        pthparts = pth.parts
        pthparts_type = pthparts[3].split("__")
        text = "Image " + pthparts_type[0] + " is classified as " + pthparts_type[1]
        t.append(text)
    df['text'] = t
    print(df['text'])
        

In [8]:
#Individual Segmentation Code
def segmentation(pathway):
    imagesl = glob.glob(pathway + '/*.png')
    df = pd.DataFrame({'Image_dir': imagesl})
    for img1 in df['Image_dir'][:3]:
        image = cv2.imread(img1)
        img = cv2.resize(image ,(240, 240))
        img = img / 255
        img = img[np.newaxis, :, :, 1]
        pred=model1.predict(img)
        plt.figure(figsize=(20,20))
        plt.subplot(1,4,1)
        plt.imshow(np.squeeze(img))
        plt.title('Original Image')
        plt.subplot(1,4,2)
        plt.imshow(np.squeeze(pred))
        plt.title('Prediction')
        plt.subplot(1,4,3)
        plt.imshow(np.squeeze(pred) > 0.5)
        plt.title('BinaryPrediction')
        plt.show()        
    

In [9]:
#Segmentation
segment_button = widgets.Button(description="Segment Images", button_style='primary')
segment_output = widgets.Output()
#display(segment_output)

#display(segment_button, segment_output)

def segment_images(b):
    with segment_output:
        clear_output(True)
        try:
            #clear_output()                                         # Button is deleted after it is clicked.
            root = Tk()
            root.withdraw()                                        # Hide the main window.
            root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
            b.files = filedialog.askopenfilename(multiple=True)    # List of selected files will be set button's file attribute.
            destination = 'C:\Voila\processing'
            del_files(destination+ '/*')
            for images in b.files:
                shutil.copy(images, destination)        
            print("All images Successfully uploaded")
            segmentation(destination)
        except:
            print("No image Uploaded")
        
segment_button.on_click(segment_images)

In [10]:
#Classification

classification_button = widgets.Button(description="Classify Images", button_style='primary')
classification_output = widgets.Output()
#display(classification_output)

#display(classification_button, classification_output)

def classify_images(b):
    with classification_output:
        clear_output(True)
        try:
            #clear_output()                                         # Button is deleted after it is clicked.
            root = Tk()
            root.withdraw()                                        # Hide the main window.
            root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
            b.files = filedialog.askopenfilename(multiple=True)    # List of selected files will be set button's file attribute.
            destination = 'C:\Voila\processing'
            del_files(destination+ '/*')
            for images in b.files:
                shutil.copy(images, destination)        
            print("All images Successfully uploaded")
            classify(destination)
        except:
            print("No image Uploaded")
        
classification_button.on_click(classify_images)

In [11]:
#Segmentation and Classification

In [12]:
##### ****************************  ##############
def prelim_class(img1):
    image = cv2.imread(img1)    
    img = cv2.resize(image ,(256, 256))
    img = img / 255
    #img = img[np.newaxis, :, :, 1]
    img = np.expand_dims(img, axis=0)
    pred=model.predict(img)
    mod_pred = np.argmax(pred, axis =1)
    if mod_pred[0] == 0:
        return "HGG"
    elif mod_pred[0] == 1:
        return "LGG"
    else:
        return "No Tumor"
    
    
def segmentation_class(pathway):
    imagesl = glob.glob(pathway + '/*.png')
    df = pd.DataFrame({'Image_dir': imagesl})
    print(df)
    for img1 in df['Image_dir'][:3]:
        pred2 = prelim_class(img1)
        pred_title = "Predicted Glioma: " + pred2
        image = cv2.imread(img1)
        img = cv2.resize(image ,(240, 240))
        img = img / 255
        img = img[np.newaxis, :, :, 1]
        pred=model1.predict(img)        
        plt.figure(figsize=(20,20))
        plt.subplot(1,4,1)
        plt.imshow(np.squeeze(img))
        plt.title('Original Image')
        plt.subplot(1,4,2)
        plt.imshow(np.squeeze(pred))
        plt.title(pred_title)
        plt.subplot(1,4,3)
        plt.imshow(np.squeeze(pred) > 0.5)
        plt.title('BinaryPrediction: ' + pred2)
        plt.show()       

In [13]:
#Segmentation
def predict_segments(pathway):
    imagesl = glob.glob(pathway + '/*.png')
    df = pd.DataFrame({'Image_dir': imagesl})
    for img1 in df['Image_dir']:        
        pth = Path(img1)
        pthparts = pth.parts
        image_name = pthparts[3]        
        image = cv2.imread(img1)
        img = cv2.resize(image ,(240, 240))
        img = img / 255
        img = img[np.newaxis, :, :, 1]
        pred=model1.predict(img)
        plt.imsave('processing/' + "seg__" + image_name, np.squeeze(pred))
        
        
upload_button = widgets.Button(description="Upload Image", button_style='info')
upload_output = widgets.Output()
#display(upload_output)
        
#display(upload_button, upload_output)

def uploads(b):
    with upload_output:
        clear_output(True)        
        try:
            #clear_output()                                         # Button is deleted after it is clicked.
            root = Tk()
            root.withdraw()                                        # Hide the main window.
            root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
            b.files = filedialog.askopenfilename(multiple=True)    # List of selected files will be set button's file attribute.
            destination = 'C:\Voila\processing'
            del_files(destination+ '/*')
            for images in b.files:
                shutil.copy(images, destination)      
            predict_segments(destination)
            print("Images Successfully Uploaded")
        except:
            print("No image Uploaded")
        
#upload_button = Button(description="Upload Images")
upload_button.on_click(uploads)



In [14]:
#prelim_class("C:\Voila\processing\HGG-Axial-BraTS19_2013_17_1_flair_sliceno-100.png")

In [15]:
#predict_segments("C:\Voila\processing\HGG-Axial-BraTS19_2013_17_1_flair_sliceno-110.png")

In [16]:
##### ****************************  ##############

def prelim_class(img1):   #This function is used to classify mri slice as HGG, LGG or No Tumor (NT)
    image = cv2.imread(img1)    
    img = cv2.resize(image ,(256, 256))
    img = img / 255    
    img = np.expand_dims(img, axis=0)
    pred=model.predict(img)
    mod_pred = np.argmax(pred, axis =1)
    if mod_pred[0] == 0:
        return "HGG"
    elif mod_pred[0] == 1:
        return "LGG"
    else:
        return "No Tumor"


def pixcnt(dfcol):       #function to calculate total number of pixels
    first_lst = []
    second_lst = []    
    for lst in dfcol:
        avg = [statistics.mean(j) for j in lst]
        first_lst.append(avg)
    for k in first_lst:
        second_lst.append(sum(k))
    return second_lst


segandclass_button = widgets.Button(description="Full Prediction", button_style='primary')
segandclass_output = widgets.Output()


src = 'C:/Voila/processing/*'

def img_dataframe(b):  #This function takes care of showing predictions via dataframe of images
    with segandclass_output:
            clear_output(True)
            path = "C:/Voila/processing/"
            directory = os.listdir(path)
            if len(directory) == 0: 
                print("Output Deleted / No Image Uploaded")
            else:
                try:
                    imagesl = glob.glob('C:/Voila/processing/*')    #search source folder and create a list of all images
                    df = pd.DataFrame({'Image_dir': imagesl}) #add images as a dataframe
                    origin = []              
                    for images in df["Image_dir"]:            
                        pth = Path(images)
                        pthparts = pth.parts
                        img_info = pthparts[3]    #extract image name
                        origin.append(img_info.replace("seg__", ""))        
                    df["img_type"] = ["seg" if img[20:23] == "seg" else "image" for img in df["Image_dir"]]  #labeling the image as to whether or not it is a segment      
                    df["origin_image"] = origin                      #create a reference column to later perform merge
                    df_seg = df.loc[df["img_type"]!= "image"] #seperating segment images from original images
                    df_img = df.loc[df["img_type"]!= "seg"] #seperating original images from segmented images
                    df_seg.columns = ['segs_dir', 'segs_imgtype', 'segs_origin_image']   #renaming columns for distinction
                    df_img.columns = ['Image_dir', 'image_type', 'image_origin'] #enaming columns for distinction
                    df = df_img.merge(df_seg, how="left", left_on="image_origin", right_on="segs_origin_image")        #merging on origin column
                    df= df[['Image_dir', 'segs_dir']]     #selecting the two most relevant columns
                    df["pred_pxls"] = [Image.open(seg_img).getdata() for seg_img in df["segs_dir"]]  #calculating pixel count
                    df["pred_pxls"] = pixcnt(df["pred_pxls"])        #calling total pixel count function
                    df = df.sort_values(by=["pred_pxls"], ascending=False)

                    for idx in range(len(df)):         
                        imagepath = os.path.join(df['Image_dir'].iloc[idx])
                        segpath = os.path.join(df['segs_dir'].iloc[idx])
                        pth = Path(imagepath)
                        pthparts = pth.parts
                        image_name = pthparts[3]
                        pred2 = prelim_class(imagepath)
                        pred_title = "Predicted Glioma: " + pred2

                        image = cv2.imread(imagepath)
                        seg = cv2.imread(segpath)

                        img = cv2.resize(image ,(240, 240))
                        img = img / 255
                        img = img[np.newaxis, :, :, 1]
                        pred=model1.predict(img)
                        plt.figure(figsize=(20,20))
                        plt.subplot(1,4,1)
                        plt.imshow(np.squeeze(img))
                        plt.title(image_name)
                        plt.subplot(1,4,2)
                        plt.imshow(np.squeeze(pred))
                        plt.title(pred_title)
                        plt.subplot(1,4,3)
                        plt.imshow(np.squeeze(pred) > 0.5)
                        plt.title('BinaryPrediction: ' + pred2)
                        plt.subplot(1,4,4)
                        plt.imshow(np.squeeze(img), cmap='gray')
                        plt.imshow(np.squeeze(pred),alpha=0.3,cmap='Reds')
                        plt.title('OverLay')
                        plt.show()
                except:
                    pass

segandclass_button.on_click(img_dataframe)
   

In [17]:
#prediction of patient samples
#Segmentation

patients_button = widgets.Button(description="Patient Upload", button_style='info')
patients_output = widgets.Output()
#display(patients_output)

#display(patients_button, patients_output)

def select_folder(b):
    with patients_output:
        try:
            clear_output(True)                                         
            root = Tk()
            root.withdraw()                                        # Hide the main window.         
            root.call('wm', 'attributes', '.', '-topmost', True)
            folder_selected = filedialog.askdirectory()   # List of selected files will be set button's file attribute.        
            pth = Path(folder_selected)
            pthparts = pth.parts
            newfolder_name = pthparts[3]
            #print(folder_selected)
            destination = 'C:/Voila/Patients'
            del_folder(destination+ '/*')
            for image_dirs in folder_selected:
                shutil.copytree(folder_selected, destination + '/' + newfolder_name) 
        except:
            print("Upload Complete")

        
patients_button.on_click(select_folder)
        
        

In [18]:
panalysis_button = widgets.Button(description="Patient Analysis", button_style='primary')
panalysis_output = widgets.Output()
#display(panalysis_output) 

#display(panalysis_button, panalysis_output)

def patient_analysis(b):
    with panalysis_output:
        clear_output(True)
        path = "C:/Voila/Patients/"
        directory = os.listdir(path)
        if len(directory) == 0: 
            print("Output Deleted / No Image Uploaded") 
        else:         
            try:            
                filelist = []    
                for root, dirs, files in os.walk("C:\Voila\Patients"):
                    for file in files:        
                        filelist.append(os.path.join(root,file))  #append the file name to the list  
                df = pd.DataFrame({'Image_dir': filelist}) #creating image dataframe
                df["Image_dir"] = [path.replace("\\", "/") for path in df["Image_dir"]]   #structuring imag path string
                folder = []
                for path in df["Image_dir"]:
                    pth = Path(path)
                    pth_part = pth.parts
                    foldername = pth_part[3] + '_' + pth_part[4] + '_' + pth_part[5]
                    folder.append(foldername)
                df["subfolder"] = folder   
                df["Image_type"] = ["scan" if img[-8:-4] == "scan" else "mask" for img in df["Image_dir"]] #add a column for image types
                df_scan = df.loc[df["Image_type"] == "scan"] #seperating scans from images
                df_mask = df.loc[df["Image_type"] == "mask"] #seperating scans from images
                df_scan.columns = ['scans_dir', 'scans_subfolder', 'scans_Image_type'] #renaming columns
                df_mask.columns = ['masks_dir', 'masks_subfolder', 'masks_Image_type'] #renaming columns
                df_img = df_scan.merge(df_mask, how="left", left_on='scans_subfolder', right_on='masks_subfolder') #aligning scans with masks
                df = df_img[['scans_dir', 'masks_dir']]
                for idx in range(len(df)):        
                    imagePath = os.path.join(df_img['scans_dir'].iloc[idx])
                    maskPath = os.path.join(df_img['masks_dir'].iloc[idx])
                    #print(imagePath)
                    pred2 = prelim_class(imagePath)
                    pred_title = "Predicted Glioma: " + pred2

                    image = cv2.imread(imagePath)
                    mask = cv2.imread(maskPath)

                    img = cv2.resize(image ,(240, 240))
                    img = img / 255
                    img = img[np.newaxis, :, :, 1]
                    pred1=model1.predict(img)
                    mask = cv2.resize(mask ,(240, 240))
                    mask = mask/255

                    plt.figure(figsize=(20,20))
                    plt.subplot(1,5,1)
                    plt.imshow(np.squeeze(img))
                    plt.title('Original Image')
                    plt.subplot(1,5,2)
                    plt.imshow(mask)
                    plt.title('Original Mask')
                    plt.subplot(1,5,3)
                    plt.imshow(np.squeeze(pred1))
                    plt.title(pred_title)
                    plt.subplot(1,5,4)
                    plt.imshow(np.squeeze(pred1) > 0.5)
                    plt.title('BinaryPrediction: ' + pred2)
                    plt.subplot(1,5,5)
                    plt.imshow(np.squeeze(img), cmap='gray')
                    plt.imshow(np.squeeze(pred1),alpha=0.3,cmap='Reds')
                    plt.title('OverLay')
                    plt.show()
            except:
                pass


panalysis_button.on_click(patient_analysis)    

In [19]:
#clear output
clear_button = widgets.Button(description="Clear Output", button_style='info')
clear_outputs = widgets.Output()
#display(clear_outputs)

#display(clear_button, clear_outputs)

def clear_images(b):
    with clear_outputs:
        clear_output(True)        
        try:
            destination1 = 'C:\Voila\processing\*'            
            imagesl = glob.glob(destination1)
            df = pd.DataFrame({'Image_dir': imagesl})        
            if len(df) >= 1:
                del_files(destination1)               
                img_dataframe(b)
            else:
                print("")            
        except:
            pass    
        

clear_button.on_click(clear_images)

In [20]:
delete_button = widgets.Button(description="Delete", button_style='info')
delete_outputs = widgets.Output()


def delete_images(b):
    with delete_outputs:
        clear_output(True)        
        try:
            destination1 = 'C:/Voila/Patients/*'            
            imagesl = glob.glob(destination1)
            df = pd.DataFrame({'Image_dir': imagesl})        
            if len(df) >= 1:
                del_folder(destination1)               
                patient_analysis(b)
            else:
                print("")            
        except:
            pass
    

delete_button.on_click(delete_images)

In [21]:
extract_results_button = widgets.Button(description="Download", button_style='primary')
extract_results_outputs = widgets.Output()

def load_results(b):    
    try:
        imagesl = glob.glob('C:/Voila/processing/*')

        df = pd.DataFrame({'Image_dir': imagesl})
        image_list = []
        classification = []
        folder = []

        for paths in df["Image_dir"]:
            pth = Path(paths)
            pthparts = pth.parts
            filename = pthparts[3].replace("seg__","")
            image_list.append(pthparts[3])
            classification.append(prelim_class(paths))    
            folder.append(filename)
        df["Image"] = image_list
        df["image_type"] = ["seg" if img[0:3] == "seg" else "image" for img in df["Image"]]
        df["folder"] = folder
        df["classification"] = classification
        dfimg = df.loc[df["image_type"] != "seg"]
        dfseg = df.loc[df["image_type"] !="image"]
        dfseg.columns = ['seg_dir', 'segimage', 'segtype', 'seg_folder', 'seg_classification']
        df1 = dfimg.merge(dfseg, left_on = 'folder', right_on= 'seg_folder', how = 'left')
        df1["pred_pxls"] = [Image.open(seg_img).getdata() for seg_img in df1["seg_dir"]]  #calculating pixel count
        df1["pred_pxls"] = pixcnt(df1["pred_pxls"])        #calling total pixel count function
        df1 = df1.sort_values(by=["pred_pxls"], ascending=False)
        df1 = df1[['Image', 'segimage', 'seg_classification', 'pred_pxls']]
        df1.columns = ['Images', 'Segmented Images', 'Classification', 'Segment Pixel Count']
        df1.to_csv('C:/Voila/data/result.csv', index=False)
        file = "C:/Voila/data/result.csv"
        os.startfile(file)
    except:
        pass
        
extract_results_button.on_click(load_results)        
    

In [22]:
#Button Creation
buttons00 = widgets.VBox([panalysis_button, panalysis_output])
buttons01 = widgets.HBox([delete_button, delete_outputs, patients_button, patients_output])
buttons1 = widgets.VBox([classification_button, classification_output])
buttons11 = widgets.VBox([segment_button, segment_output])
#buttons2 = widgets.HBox([clear_button, clear_outputs])
#buttons02 = widgets.HBox([delete_button, delete_outputs])
buttons03 = widgets.HBox([clear_button, clear_outputs, upload_button, upload_output,])
buttons3 = widgets.VBox([extract_results_button, segandclass_button, segandclass_output,])
#buttons4 = widgets.VBox([extract_results_button, extract_results_outputs])
#buttons0 = widgets.VBox([panalysis_button, patients_button, panalysis_output, patients_output, \
                         #classification_button, segment_button, classification_output, segment_output, \
                        #clear_button, delete_button, clear_outputs, delete_outputs, \
                        #upload_button, segandclass_button, upload_output, segandclass_output])

In [23]:
#display(buttons03)

In [24]:
########################################################################################################################################################################

In [25]:
#N Grams Data Frames
df_grams = pd.read_csv("C:/Voila/NLP_Extraction/Actual/Paper_Details/grams.csv",  index_col=0)


In [26]:
#keywords Dataframe
df_kw = pd.read_excel("C:/Voila/NLP_Extraction/Actual/Paper_Details/glioma_keywords.xlsx", index_col=0)
df_kw["expert_word_cnt"] = np.where(df_kw["expert_labels"]=="*****", 0, df_kw["expert_word_cnt"])
df_kw["expert_labels"] = np.where(df_kw["expert_labels"]=="*****", "None", df_kw["expert_labels"])


In [27]:
#naming Strings and values
df_kw["expert_word_cnt"] = np.where(df_kw["expert_labels"]=="*****", 0, df_kw["expert_word_cnt"])
df_kw["expert_labels"] = np.where(df_kw["expert_labels"]=="*****", "None", df_kw["expert_labels"])

In [28]:
#creating expert words df using title, year, expert_labels
df_exp = df_kw.groupby(['title', 'Year', 'expert_labels'], as_index=False)['expert_word_cnt'].mean()


In [29]:
#creating main words df using title, year, main_words
#instead of plotting main words, n grams will be plotted instead
df_mw = df_kw.groupby(['title', 'Year', 'main_words'], as_index=False)['main_word_cnt'].mean()

In [30]:
title_dropdown = widgets.Dropdown(
    options=list(df_exp['title'].unique()),
    value='A histopathological diagnostic marker for human spinal astrocytoma_ expression of glial fibrillary acidic protein-δ.pdf',
    description='Documents',
)

In [31]:
#Allow widget to interact with any plot

year_widget0 = widgets.IntRangeSlider(
    value=[2003, 2007],
    min=2010,
    max=2020,
    step=1.0,
    description='Years:',
    continuous_update=False
)

exp_output0 = widgets.Output()

year_widget1 = widgets.IntRangeSlider(
    value=[2003, 2007],
    min=2010,
    max=2020,
    step=1.0,
    description='Years:',
    continuous_update=False
)

exp_output1 = widgets.Output()

l1 = widgets.link((year_widget1, 'value'), (year_widget0, 'value'))


@ipywidgets.interact(year=year_widget0)
def expwd_plot(year):    
    with exp_output1:
        clear_output(True)
        try:
            df_exp0 = df_exp.groupby(['title','Year', 'expert_labels'], as_index=False)['expert_word_cnt'].sum()
            df_exp1 = df_exp0.sort_values(by=['Year'])
            dframe = df_exp1[df_exp1['Year'].between(year[0], year[1])]
            df_00 = df_exp.groupby(['Year', 'expert_labels'], as_index=False)['expert_word_cnt'].sum()
            dframe.columns = ['title', 'Year', 'expert_labels', 'Expert Word Amount']
            fig = px.bar(data_frame=dframe, x="Year", y="Expert Word Amount", color='expert_labels', barmode='group')#, text='Expert Word Amount')
            fig.update_traces(textposition='outside')
            fig.update_layout(
            height=500,
            title_text='Total Expert Word Count per Document(2010 - 2021)')
            year_widget0.layout.display = 'none'
            fig.show()
        except:
            pass

interactive(children=(IntRangeSlider(value=(2010, 2010), continuous_update=False, description='Years:', max=20…

In [32]:
#Allow widget to interact with any plot
grams_dropdown0 = widgets.Dropdown(
    options=list(df_grams['Grams'].unique()),
    value='Unigram',
    description='N-Grams',
)
grams_output0 = widgets.Output()

grams_dropdown1 = widgets.Dropdown(
    options=list(df_grams['Grams'].unique()),
    value='Unigram',
    description='N-Grams',
)

l = widgets.link((grams_dropdown1, 'value'), (grams_dropdown0, 'value'))

grams_output1 = widgets.Output()

@ipywidgets.interact(gram=grams_dropdown0)
def expwd_plot(gram):    
    with grams_output1:
        clear_output(True)
        try:
            dframe = df_grams.loc[df_grams["Grams"] == gram]
            dframe.columns = ['Word-Grams', 'Frequency', 'Grams']
            fig = px.bar(data_frame=dframe, x="Word-Grams", y="Frequency", color='Grams', barmode='group', text='Frequency')
            fig.update_traces(textposition='outside')
            fig.update_layout(
            height=500,
            title_text='Word Contiguous Sequences')
            grams_dropdown0.layout.display = 'none'
            fig.show()
        except:
            pass

interactive(children=(Dropdown(description='N-Grams', options=('Unigram', 'Bigram', 'Trigram'), value='Unigram…

In [33]:
year_slider = widgets.VBox([year_widget1, exp_output1])
grams_select = widgets.VBox([grams_dropdown1, grams_output1])

In [34]:
tab1 = VBox(children=[buttons01, buttons00,])
tab2 = VBox(children=[buttons03, buttons3,])
tab3 = VBox(children=[year_slider,])
tab4 = VBox(children=[grams_select,])

In [35]:
tab = widgets.Tab(children=[tab1, tab2, tab3, tab4, ])
tab.set_title(0, 'Patient Analysis')
tab.set_title(1, 'Prediction Tool')
tab.set_title(2, 'Expert Word Analysis')
tab.set_title(3, 'N-Grams')
VBox(children=[tab])

#### Jermaine Samuels ######
Msc Applied Data Science
ID: 6200005491